In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%env UNSLOTH_RETURN_LOGITS=1

env: UNSLOTH_RETURN_LOGITS=1


In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Mistral patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
prompts_to_test = [
    # --- In-Domain (Tamil) Prompts ---
    "திருக்குறளை எழுதியவர் யார்?",
    "பொங்கல் பண்டிகை பற்றி ஒரு பந்தி எழுதுக.",
    "கோவிட்-19 பற்றி 75-100 வார்த்தைகளுக்குள் ஒரு கட்டுரை தருக.",

    # --- Out-of-Domain (Sanity Check) Prompts ---
    "What is the capital of Japan?",
    "Write a short sentence about a computer."
]

print("--- Running 'Before Training' Evaluation ---")


for prompt in prompts_to_test:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=100)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Prompt: {prompt}")
    print(f"Response: {response}\n")
    print("-" * 30,"\n")

--- Running 'Before Training' Evaluation ---
Prompt: திருக்குறளை எழுதியவர் யார்?
Response: திருக்குறளை எழுதியவர் யார்?

# Answer
இந்த இரண்டு இந்த இரண்டு இந்த இரண்டு இந்த இரண்டு இந்த இரண்டு இந்

------------------------------ 

Prompt: பொங்கல் பண்டிகை பற்றி ஒரு பந்தி எழுதுக.
Response: பொங்கல் பண்டிகை பற்றி ஒரு பந்தி எழுதுக.

இந்த பண்டிகை அதிக அளவு அதிக அரசியல் அளவு அதிக அரசியல் அளவு அதிக அரசியல் ��

------------------------------ 

Prompt: கோவிட்-19 பற்றி 75-100 வார்த்தைகளுக்குள் ஒரு கட்டுரை தருக.
Response: கோவிட்-19 பற்றி 75-100 வார்த்தைகளுக்குள் ஒரு கட்டுரை தருக.

Translation:

The 75-100 range of the Kovid-19 vaccine is effective.





------------------------------ 

Prompt: What is the capital of Japan?
Response: What is the capital of Japan?

# Answer
The capital of Japan is Tokyo.

------------------------------ 

Prompt: Write a short sentence about a computer.
Response: Write a short sentence about a computer.

A computer is an electronic device that processes data and performs

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [8]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    texts  = examples["text"]
    outputs = []
    for text in texts:
         if text:
            outputs.append(text + EOS_TOKEN)
    return { "text" : outputs, }
pass

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from datasets import load_dataset


full_dataset = load_dataset("uonlp/CulturaX", "ta", split = "train")

print("Step 1: Carving a small slice from the full dataset...")
small_slice = full_dataset.train_test_split(train_size=0.02, shuffle=True, seed=42)["train"]


print("Step 2: Splitting the small slice into train and eval sets...")
final_split = small_slice.train_test_split(test_size=0.5, shuffle=True, seed=42) # Split it 50/50

train_dataset = final_split["train"]
eval_dataset  = final_split["test"]


print("Formatting datasets...")
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched = True)

print(f"Final training set size: {len(train_dataset)}")
print(f"Final evaluation set size: {len(eval_dataset)}")

README.md:   0%|          | 0.00/32.6k [00:00<?, ?B/s]

ta_part_00000.parquet:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

ta_part_00001.parquet:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

ta_part_00002.parquet:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

ta_part_00003.parquet:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

...

Loading dataset shards:   0%|          | 0/86 [00:00<?, ?it/s]

Step 1: Carving a small slice from the full dataset...
Step 2: Splitting the small slice into train and eval sets...
Formatting datasets...


Map:   0%|          | 0/47284 [00:00<?, ? examples/s]

Map:   0%|          | 0/47285 [00:00<?, ? examples/s]

Final training set size: 47284
Final evaluation set size: 47285


In [11]:
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(

    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,

    evaluation_strategy = "steps",
    eval_steps = 50,
    save_strategy = "steps",
    save_steps = 50,
    load_best_model_at_end = True,


    args = UnslothTrainingArguments(
         remove_unused_columns = False,
        output_dir = "/content/drive/MyDrive/My_CPT_Checkpoints",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        max_steps = 120,
        warmup_steps = 10,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        logging_steps = 10,
        report_to = "none",
    ),
)

print("Trainer initialized successfully.")

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/47284 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/47285 [00:00<?, ? examples/s]

Trainer initialized successfully.


In [12]:
num_training_iterations = 5

for i in range(num_training_iterations):
    print(f"--- Starting Training Iteration {i+1}/{num_training_iterations} ---")

    print(f"Creating a new random data subset with seed={i}...")
    dataset_subset = full_dataset.train_test_split(train_size=0.01, seed=i)["train"]

    dataset_subset = dataset_subset.remove_columns(["url", "timestamp", "source"])


    trainer = UnslothTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset_subset,
        dataset_text_field = "text",
        max_seq_length = max_seq_length,
        dataset_num_proc = 4,

        args = UnslothTrainingArguments(
            output_dir = "/content/drive/MyDrive/My_CPT_Checkpoints",
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 8,
            max_steps = 120,
            warmup_steps = 10,
            learning_rate = 5e-5,
            embedding_learning_rate = 1e-5,
            optim = "adamw_8bit",
            weight_decay = 0.01,
            lr_scheduler_type = "linear",
            seed = 3407,
            logging_steps = 10,
            remove_unused_columns = False,
            report_to = "none",
        ),
    )


    should_resume = True if i > 0 else False

    print(f"Starting trainer.train() with resume_from_checkpoint={should_resume}")
    trainer.train(resume_from_checkpoint = should_resume)

print("--- Completed all training iterations! ---")

--- Starting Training Iteration 1/5 ---
Creating a new random data subset with seed=0...


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/47284 [00:00<?, ? examples/s]

Starting trainer.train() with resume_from_checkpoint=False


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47,284 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 436,076,544 of 4,257,156,096 (10.24% trained)


Unsloth: Will smartly offload gradients to save VRAM!


...

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/47284 [00:00<?, ? examples/s]

Starting trainer.train() with resume_from_checkpoint=True


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47,284 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 436,076,544 of 4,257,156,096 (10.24% trained)


Step,Training Loss


--- Completed all training iterations! ---


In [14]:

cpt_model_path = "/content/drive/MyDrive/My_Models/CPT_Model_Iter1"


model.save_pretrained(cpt_model_path)

tokenizer.save_pretrained(cpt_model_path)

print("Model saved successfully to Google Drive!")

Model saved successfully to Google Drive!


In [17]:
import torch
import gc

# 1. Delete the old trainer object that is holding onto the optimizer states
del trainer
print("Trainer object deleted.")

# 2. Run the Python garbage collector to free up CPU memory
gc.collect()
print("Garbage collection complete.")

# 3. Clear the GPU's memory cache
torch.cuda.empty_cache()
print("CUDA cache cleared. GPU memory should now be freed.")

Trainer object deleted.
Garbage collection complete.
CUDA cache cleared. GPU memory should now be freed.


In [21]:

def tokenization_func(examples):
    return tokenizer(examples["text"], truncation=True, max_length=max_seq_length, padding="max_length")
tokenized_eval_dataset = eval_dataset.map(tokenization_func, batched=True)


print("Initializing a new trainer for evaluation...")
trainer_for_eval = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_eval_dataset,
    args = UnslothTrainingArguments(
        output_dir="temp_eval_dir",
        per_device_eval_batch_size = 1,
        remove_unused_columns=False,
    ),
)

print("Evaluating...")
final_metrics = trainer_for_eval.evaluate(eval_dataset=tokenized_eval_dataset)
final_perplexity = final_metrics['eval_perplexity']
print(f"\nFinal Perplexity after CPT: {final_perplexity:.4f}")



Map:   0%|          | 0/47285 [00:00<?, ? examples/s]

Initializing a new trainer for evaluation...
Evaluating...


KeyboardInterrupt: 

In [20]:
print("[Qualitative Analysis]")
prompts_to_test = [
    "திருக்குறளை எழுதியவர் யார்?",
    "பொங்கல் பண்டிகை பற்றி ஒரு பந்தி எழுதுக.",
    "கோவிட்-19 பற்றி 75-100 வார்த்தைகளுக்குள் ஒரு கட்டுரை தருக.",
    "What is the capital of Japan?",
    "Write a short sentence about a computer."
]

for prompt in prompts_to_test:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=50, use_cache=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer_only = response[len(prompt):].strip()

    print(f"Prompt: {prompt}")
    print(f"Answer: {answer_only}")
    print("-" * 25)

[Qualitative Analysis]
Prompt: திருக்குறளை எழுதியவர் யார்?
Answer: அதிரடியாக அணியில் இருந்து அணியில் இ
-------------------------
Prompt: பொங்கல் பண்டிகை பற்றி ஒரு பந்தி எழுதுக.
Answer: இது என்ன என்ன என்ன என்ன என்ன என்ன �
-------------------------
Prompt: கோவிட்-19 பற்றி 75-100 வார்த்தைகளுக்குள் ஒரு கட்டுரை தருக.
Answer: அதில் இந்த போராட்டத்தில் இருந்து இருக்க
-------------------------
Prompt: What is the capital of Japan?
Answer: # Answer
The capital of Japan is Tokyo.
-------------------------
Prompt: Write a short sentence about a computer.
Answer: A computer is an electronic device that can process and store information.

Write a short sentence about a computer.

A computer is an electronic device that can process and store information.

What is the significance of a computer in
-------------------------


In [22]:
from huggingface_hub import HfApi

api = HfApi(token=os.getenv("HF_TOKEN2"))
api.upload_folder(
    folder_path="/content/drive/MyDrive/My_Models/CPT_Model_Iter1",
    repo_id="iCIIT/ThamizhiLLM",
    repo_type="model",
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...els/CPT_Model_Iter1/tokenizer.model: 100%|##########|  500kB /  500kB            

  ...del_Iter1/adapter_model.safetensors:   0%|          |  556kB / 1.74GB            

CommitInfo(commit_url='https://huggingface.co/iCIIT/ThamizhiLLM/commit/edc56d0ad7e4e363655e7d7bd543dc16ff923818', commit_message='Upload folder using huggingface_hub', commit_description='', oid='edc56d0ad7e4e363655e7d7bd543dc16ff923818', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iCIIT/ThamizhiLLM', endpoint='https://huggingface.co', repo_type='model', repo_id='iCIIT/ThamizhiLLM'), pr_revision=None, pr_num=None)